In [ ]:
from pyspark import SparkContext

def run_pagerank(file_path, is_sanity_check=False):
    sc = SparkContext(appName="PageRank")
    beta = 0.8
    iterations = 40

    # Load and parse edges (deduplicate)
    data = sc.textFile(file_path)
    edges = data.map(lambda line: tuple(map(int, line.strip().split()))).distinct()

    # Get all nodes
    nodes = edges.flatMap(lambda x: x).distinct()
    n = nodes.count()

    # Adjacency list and out-degrees
    outlinks = edges.groupByKey().mapValues(set).cache()
    out_degrees = outlinks.mapValues(len).collectAsMap()
    out_degree_broadcast = sc.broadcast(out_degrees)

    # Initial PageRank vector
    ranks = nodes.map(lambda node: (node, 1.0 / n)).cache()

    for _ in range(iterations):
        contribs = outlinks.join(ranks).flatMap(
            lambda node_data: [
                (dst, node_data[1][1] / out_degree_broadcast.value[node_data[0]])
                for dst in node_data[1][0]
            ]
        )
        ranks = contribs.reduceByKey(lambda x, y: x + y).mapValues(
            lambda sum_rank: ((1 - beta) / n) + beta * sum_rank
        ).cache()

    # Collect results
    final_ranks = ranks.collect()
    top_10 = sorted(final_ranks, key=lambda x: -x[1])[:10]
    bottom_10 = sorted(final_ranks, key=lambda x: x[1])[:10]

    # Output
    print(f"\nPageRank Results for: {file_path}")
    print("Top 10 nodes by PageRank:")
    for node, score in top_10:
        print(f"Node {node}: {score:.6f}")

    print("\nBottom 10 nodes by PageRank:")
    for node, score in bottom_10:
        print(f"Node {node}: {score:.6f}")

    # Optional sanity check
    if is_sanity_check:
        top_node = top_10[0]
        print(f"\nSanity Check — Top Node ID: {top_node[0]}, Score: {top_node[1]:.6f}")
        print("Expected: Node 53 with score ~0.036")

    sc.stop()

# ---------- Run Either Dataset ----------
# Run with graph-full.txt
run_pagerank("graph-full.txt")

# Run with graph-small.txt (sanity check)
run_pagerank("graph-small.txt", is_sanity_check=True)



PageRank Results for: graph-full.txt
Top 10 nodes by PageRank:
Node 263: 0.002020
Node 537: 0.001943
Node 965: 0.001925
Node 243: 0.001853
Node 285: 0.001827
Node 255: 0.001802
Node 126: 0.001801
Node 502: 0.001800
Node 16: 0.001792
Node 747: 0.001785

Bottom 10 nodes by PageRank:
Node 558: 0.000329
Node 93: 0.000351
Node 62: 0.000353
Node 424: 0.000355
Node 408: 0.000388
Node 742: 0.000393
Node 81: 0.000407
Node 920: 0.000411
Node 657: 0.000428
Node 386: 0.000432

PageRank Results for: graph-small.txt
Top 10 nodes by PageRank:
Node 53: 0.035731
Node 14: 0.034171
Node 40: 0.033630
Node 1: 0.030006
Node 27: 0.029720
Node 66: 0.029195
Node 48: 0.025397
Node 79: 0.019613
Node 61: 0.019180
Node 65: 0.019117

Bottom 10 nodes by PageRank:
Node 85: 0.003410
Node 59: 0.003670
Node 81: 0.003695
Node 37: 0.003808
Node 89: 0.003922
Node 84: 0.004036
Node 23: 0.004124
Node 94: 0.004248
Node 50: 0.004258
Node 7: 0.004274

Sanity Check — Top Node ID: 53, Score: 0.035731
Expected: Node 53 with score